# Load Packages

What we can recall from critisicm regarding this presentation:

- did not take the equal weighted as benchmark
- use total return indices (I think)

In [1]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd

In [2]:
# Seaborn
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

# little function for later
def isNaN(string):
    return string != string

# Load Data

In [80]:
# rawdata is direct import from excel

# Import of excel data, sheet by sheet
xls_ETF = pd.ExcelFile("1_Data/Data_ETF_YIELDS_1.0.xlsx")
xls_inflation = pd.ExcelFile("1_Data/Data_INFLATION_1.0.xlsm")

ETF_raw = pd.DataFrame(pd.read_excel(xls_ETF, 1))
Yield_raw = pd.DataFrame(pd.read_excel(xls_ETF, 2))

Yield_short_raw = pd.DataFrame(pd.read_excel(xls_ETF, 3))

cpicore_raw = pd.DataFrame(pd.read_excel(xls_inflation, 0))
targetrates_raw = pd.DataFrame(pd.read_excel(xls_inflation, 1))
#rawdata.columns = ["Trading_day", "Date", "MSCI_EU", "Healthcare", "Finacials", "Consumer_staples","Industrials","Consumer_discretionary","Materials", "Information_technology", "Energy", "Utilities", "Communication_services","Real_estate"]


# keep a safe copy of the rawdata to compare the changes
ETF = ETF_raw.copy()
Yield = Yield_raw.copy()
Yield_short = Yield_short_raw.copy()
cpicore = cpicore_raw.copy()
targetrates = targetrates_raw.copy()

In [81]:
# Drop all the columns in which only text is and safe them separately --> ONLY RUN ONCE
ETF_text = ETF.iloc[0,:]
Yield_text = Yield.iloc[0,:]
Yield_short_text = Yield_short.iloc[0,:]
cpicore_text = cpicore.iloc[0:1,:]


ETF.drop([0], inplace = True)
Yield.drop([0], inplace = True)
Yield_short.drop([0], inplace = True)
cpicore.drop([0,1], inplace = True)

# Set date as index
ETF.set_index("Dates", inplace=True)
Yield.set_index("Dates", inplace=True)
Yield_short.set_index("Dates", inplace=True)
cpicore.set_index("Dates", inplace=True)
targetrates.set_index("Dates", inplace=True)

# Value

Recent infl – the average of the headline and core annual CPI inflation rate
--> we don't have headline inflation


Effective target infl - The effective inflation target is the mean of the target range announced or implied by the authorities plus an adjusted for past “target misses”, which is the last 3 years’ average gap between actual inflation and the target means
--> we don't have target rates for all countries --> use 2.5 as target rate

Formula: (1/n)*recent infl + ((n-1)/n)*effective target infl

In [5]:
# create the rolling 3-year average of the core inflation

# first I have to set up an empty dictionary to store the rolling averages
cpicore_avg = {}
for i in cpicore.columns:
    cpicore_avg[i] = []

# next up we iterate over the cpicore data to get the index and safe them in the dictionary
indices_reverse = cpicore.index[::-1]
runner = len(indices_reverse)
for i in indices_reverse:
    if runner == 35:
        break
    
    for j,k in enumerate(cpicore.columns):
        inflation = np.nanmean(cpicore.iloc[runner-36:runner, j])
        cpicore_avg[k].append(inflation)
    runner -= 1
    
#cpicore_avg
cpicore_avg_trimmed = pd.DataFrame(cpicore_avg)

<ipython-input-5-526646c4995a>:16: RuntimeWarning: Mean of empty slice
  inflation = np.nanmean(cpicore.iloc[runner-36:runner, j])


In [6]:
# now calculate the difference between 3-year average and target inflation

eff_target_inf = dict(cpicore_avg)

# special cases (Australia, Canada, UK, Japan, US, Switzerland) for which we have an actual target
for key in cpicore_avg.keys():
    if key == "Australia":
        for i,k in enumerate(cpicore_avg[key]):
            if k >= 2.0 and k <= 3.0:
                eff_target_inf[key][i] = 0
            elif k < 2.0:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 3
                
    elif key == "Canada":
        for i,k in enumerate(cpicore_avg[key]):
            if k >= 1.0 and k <= 3.0:
                eff_target_inf[key][i] = 0
            elif k < 2.0:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 1
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 3
                
    elif key == "United Kingdom":
        for i,k in enumerate(cpicore_avg[key]):
            if k == 2.0:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
                
    elif key == "Japan":
        for i,k in enumerate(cpicore_avg[key]):
            if k == 3.3:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 3.3
                
    elif key == "United States":
        for i,k in enumerate(cpicore_avg[key]):
            if k == 2.0:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
                
    elif key == "Switzerland":
        for i,k in enumerate(cpicore_avg[key]):
            if k >= 0.0 and k <= 2.0:
                eff_target_inf[key][i] = 0
            elif k < 0.0:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2
                
    else:
        for i,k in enumerate(cpicore_avg[key]):
            if k == 2.5:
                eff_target_inf[key][i] = 0
            else:
                eff_target_inf[key][i] = eff_target_inf[key][i] - 2.5


In [7]:
# now we calculate the value formula:
# (1/n)*recent inflation + ((n-1)/n)*effective target inflation for the years: 2, 5, 10, 20, 30

# first we need to align the dataframes 
cpicore_trimmed = cpicore[35:]
#cpicore_avg_trimmed = pd.DataFrame(cpicore_avg)
cpicore_avg_trimmed.set_index(cpicore_trimmed.index, inplace = True)

# second we create an empty dictionary to fill with the calculations
formula_values = {}
x = 0
for i in cpicore_trimmed.columns:
    x += 1
    for j in [2,5,10,20,30]:
        formula_values[f"{x},{j}"] = []
#formula_values

# now that we have the empty dictionary to fill, we start with caluclating the inflation expectation
keys = list(formula_values.keys())

runner = 0
for i,k in enumerate (cpicore_trimmed.columns):
    for j in [2,5,10,20,30]:
        formula_values[keys[runner]].append((1/j)*cpicore_trimmed[k] +((j-1)/j)*cpicore_avg_trimmed[k])
        runner += 1

In [8]:
# in a next step we adjust the yield data to have the same lookback as the other data

Yield_trimmed = Yield[35:]

Real_yield = pd.DataFrame(columns = Yield_trimmed.columns, index = Yield_trimmed.index)

# here we calculate the difference between the yield etf and the inflation
for i,k in enumerate(Real_yield.columns):
    key = keys[i]
    for j,l in enumerate(Real_yield.index):
        Real_yield.iloc[j,i] = Yield_trimmed.iloc[j,i] - formula_values[key][0][j]

In [9]:
# in a final step we will now take the average of the countries different maturities yields

countries = Real_yield.columns
avg_yield_countries = {}
for i in range(0,len(countries),5):
    avg_yield_countries[countries[i]] = []
    
countries = list(avg_yield_countries.keys())


for i,k in enumerate(avg_yield_countries.keys()):
    for j,l in enumerate(Real_yield.index):
        m = 0
        for f in range(0,5):
            m = m + 1/5 * Real_yield.iloc[j,f+5*i]
        avg_yield_countries[countries[i]].append(m)

avg_yield_countries = pd.DataFrame(avg_yield_countries)
#avg_yield_countries

# finally we have to search for the maximum in each row

maximum_value = avg_yield_countries.idxmax(axis=1)
#maximum_value

In [15]:
maximum_value

0             UK
1             UK
2             UK
3             UK
4             UK
         ...    
240    Australia
241    Australia
242    Australia
243    Australia
244    Australia
Length: 245, dtype: object

# Momentum

In [17]:
# we start off by copying the dataframe and initiating a new empty dataframe to store the return

ETF_tr = ETF.copy()
ETF_returns = pd.DataFrame(columns = ETF_tr.columns, index = ETF_tr.index)

for i,k in enumerate(ETF_tr.index):
    if i == 0:
        continue
    for j,m in enumerate(ETF_tr.columns):
        ETF_returns.iloc[i,j] = (ETF_tr.iloc[i,j]/ETF_tr.iloc[i-1,j])-1


# here we convert the values to numeric in order to take the maximum afterwards
ETF_returns = ETF_returns.apply(pd.to_numeric)

# finally we search for the country with the highest momentum and store the values
maximum_mom = ETF_returns.idxmax(axis = 1, skipna = True)
#maximum_mom

# Carry

In [18]:
# for carry we will now calculate the differences between the long-term yields (2,5,10,20,30) and short-term yields (1M, 3M)
# we will create a dataframe for the 1M and the 3M

Carry_1m = pd.DataFrame(columns = Yield.columns, index = Yield.index)
Carry_3m = pd.DataFrame(columns = Yield.columns, index = Yield.index)
avg_Carry_1m = pd.DataFrame(0, columns = countries, index = Yield.index)
avg_Carry_3m = pd.DataFrame(0, columns = countries, index = Yield.index)

# next we will iterate through all the columns of the yields and substract the short-term rate
runner = 0
counter = 0
for i,k in enumerate(Yield.columns):
    if counter == 5:
        counter = 0
        runner += 2
    counter += 1
    Carry_1m.iloc[:,i] = Yield.iloc[:,i] - Yield_short.iloc[:,runner]
    Carry_3m.iloc[:,i] = Yield.iloc[:,i] - Yield_short.iloc[:,runner+1]
    
# next we will equal weight the carry signal among the countries
for i in range(len(countries)):
    j = 0
    while j != 5:
        avg_Carry_1m.iloc[:,i] += 1/5 * Carry_1m.iloc[:,i+j]
        avg_Carry_3m.iloc[:,i] += 1/5 * Carry_3m.iloc[:,i+j]
        j += 1
        
# change data values from object to float
avg_Carry_1m = avg_Carry_1m.apply(pd.to_numeric)
avg_Carry_3m = avg_Carry_3m.apply(pd.to_numeric)

In [83]:
# for carry we will now calculate the differences between the long-term yields (2,5,10,20,30) and short-term yields (1M, 3M)
# we will create a dataframe for the 1M and the 3M

Carry_1m = pd.DataFrame(columns = Yield.columns, index = Yield.index)
Carry_3m = pd.DataFrame(columns = Yield.columns, index = Yield.index)
avg_Carry_1m = pd.DataFrame(0, columns = countries, index = Yield.index)
avg_Carry_3m = pd.DataFrame(0, columns = countries, index = Yield.index)

# next we will iterate through all the columns of the yields and substract the short-term rate
runner = 0
counter = 0
for i,k in enumerate(Yield.columns):
    if counter == 5:
        counter = 0
        runner += 2
    counter += 1
    Carry_1m.iloc[:,i] = Yield.iloc[:,i] - Yield_short.iloc[:,runner]
    Carry_3m.iloc[:,i] = Yield.iloc[:,i] - Yield_short.iloc[:,runner+1]
    
# next we will equal weight the carry signal among the countries
Carry_1m.iloc[2:66,30:35] = 0

runner = 0
for i in range(0,len(Carry_1m.columns)-1, 5):
    avg_Carry_1m.iloc[:,runner] = np.nanmean([Carry_1m.iloc[:,i], Carry_1m.iloc[:,i+1], Carry_1m.iloc[:,i+2], Carry_1m.iloc[:,i+3], Carry_1m.iloc[:,i+4]], axis = 0)
    avg_Carry_3m.iloc[:,runner] = np.nanmean([Carry_3m.iloc[:,i], Carry_3m.iloc[:,i+1], Carry_3m.iloc[:,i+2], Carry_3m.iloc[:,i+3], Carry_3m.iloc[:,i+4]], axis = 0)
    runner += 1
    
    
    
# change data values from object to float
avg_Carry_1m = avg_Carry_1m.apply(pd.to_numeric)
avg_Carry_3m = avg_Carry_3m.apply(pd.to_numeric)

ZeroDivisionError: division by zero

In [85]:
Carry_1m.iloc[2:66,30:35] = 1
Yield.iloc[2:80,30:35]

,China,China.1,China.2,China.3,China.4
Dates,,,,,
2000-02-29,NaN,NaN,NaN,NaN,NaN
2000-03-31,NaN,NaN,NaN,NaN,NaN
2000-04-28,NaN,NaN,NaN,NaN,NaN
2000-05-31,NaN,NaN,NaN,NaN,NaN
2000-06-30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2006-03-31,2.048,2.458,3.308,4.113,NaN
2006-04-28,2.04,2.552,2.971,4.113,NaN
2006-05-31,1.954,2.5,2.982,4.113,NaN


In [53]:
x = np.nanmean([Carry_1m.iloc[0:2,0], Carry_1m.iloc[0:2,1]], axis = 0)
x


array([1.3345000000000002, 1.4124999999999996], dtype=object)

In [88]:
Carry_1m.iloc[0:2,0].sub(np.nan, fill_value = 0)

Dates
1999-12-31   NaN
2000-01-31   NaN
Name: Germany, dtype: float64

# Sort function to create the buckets

In [ ]:
def pmp_sort(pandas_DataFrame, nMax):
    top_bucket = {}
    bottom_bucket = {}
    for i,k in enumerate(pandas_DataFrame.index):
        top_bucket[k] = []
        bottom_bucket[k] = []
        a = pandas_DataFrame.iloc[i]
        for j in range(5):
            x = a.idxmax()
            y = a.idxmin()
            if isNaN(x) and isNaN(y):
                break
            top_bucket[k].append(x)
            bottom_bucket[k].append(y)
            a = a.drop([x,y])

In [23]:
top_bucket = {}
bottom_bucket = {}
for i,k in enumerate(avg_yield_countries.index):
    top_bucket[k] = []
    bottom_bucket[k] = []
    a = avg_yield_countries.iloc[i]
    for i in range(3):
        x = a.idxmax()
        y = a.idxmin()
        if isNaN(x):
            continue
        else:
            top_bucket[k].append(x)
            a = a.drop(x)
        if isNaN(y):
            continue
        else:
            bottom_bucket[k].append(y)
            a = a.drop(y)

In [24]:
print(top_bucket)
print(bottom_bucket)

# avg_yield_countries

{0: ['UK', 'Germany', 'Canada'], 1: ['UK', 'Germany', 'Canada'], 2: ['UK', 'Canada', 'Germany'], 3: ['UK', 'Canada', 'Italy'], 4: ['UK', 'Canada', 'Italy'], 5: ['UK', 'Canada', 'Italy'], 6: ['UK', 'Italy', 'Canada'], 7: ['UK', 'Italy', 'Canada'], 8: ['UK', 'Italy', 'Canada'], 9: ['UK', 'Canada', 'Italy'], 10: ['UK', 'Canada', 'Italy'], 11: ['UK', 'Canada', 'Italy'], 12: ['UK', 'Canada', 'Italy'], 13: ['UK', 'Canada', 'Italy'], 14: ['UK', 'Canada', 'Italy'], 15: ['UK', 'Canada', 'Italy'], 16: ['UK', 'Canada', 'Italy'], 17: ['UK', 'Canada', 'Italy'], 18: ['UK', 'Canada', 'Italy'], 19: ['UK', 'Canada', 'Italy'], 20: ['UK', 'Canada', 'Italy'], 21: ['UK', 'Canada', 'Italy'], 22: ['UK', 'Canada', 'Italy'], 23: ['UK', 'Canada', 'Italy'], 24: ['UK', 'Canada', 'Italy'], 25: ['UK', 'Canada', 'Italy'], 26: ['UK', 'Canada', 'Italy'], 27: ['UK', 'Canada', 'Italy'], 28: ['UK', 'Canada', 'Italy'], 29: ['UK', 'Canada', 'Italy'], 30: ['UK', 'Canada', 'Italy'], 31: ['UK', 'Canada', 'Italy'], 32: ['UK', 

In [ ]:
x = avg_Carry_1m.iloc[0].drop(["Switzerland", np.nan])
avg_Carry_1m.iloc[0]
x

In [ ]:
a = avg_yield_countries.iloc[0,2]
b = avg_yield_countries.iloc[0,4]
np.isnan(["a",b]).all()

In [ ]:
p = []
p.append(np.nan)
p

# Cut Momentum Frame

In [ ]:
# Exclude Trading Days column and MSCI_EU Benchmark to have only Momentum columns
prices = df.loc[:,"Healthcare":"Real_estate"]
prices.dtypes

In [ ]:
# Calculating Monthly returns
mtl_ret = prices.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
mtl_ret

In [ ]:
past_11 = (mtl_ret+1).rolling(11).apply(np.prod)-1 # accumulate returns over 11 months 
past_11.head(11)

In [ ]:
# Defining formation date <-  where portfolio gets created
formation = dt.datetime(1999, 12, 31, 0, 0)

In [ ]:
# Defining MeasurementDate <- up to last date of month before formation date
end_measurement = formation - MonthEnd(1)
end_measurement

In [ ]:
# Past 12 Month performance without most recent month <- in this case Jan, Feb, Mar, Apr, Jun, Jul, Sep, Oct, Nov but not December!
ret_12 = past_11.loc[end_measurement]
ret_12 = ret_12.reset_index()
ret_12.rename(columns={ ret_12.columns[1]: "returns" }, inplace = True)
ret_12

In [ ]:
ret_12 = ret_12.sort_values(["returns"], ascending=False)
ret_12["bucket"] = [2,2,2,2,2,1,0,0,0,0,0] # Make three buckets
ret_12

In [ ]:
# Create winners and loser list based on bucket
winners = ret_12[ret_12.bucket == 2]
losers = ret_12[ret_12.bucket == 0]
winners

In [ ]:
# Calculate the winner and loser returns over the last 12 months excluding the most recent month
winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners["index"])]
loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers["index"])]

In [ ]:
Momentumprofit = winnerret.mean() - loserret.mean()
Momentumprofit

# Functionalising this Methodology

In [ ]:
def data_cleaner(pandas_df, index="Date", prices_start='Healthcare', prices_end='Real_estate', resample_freq='M', lookback=11):
    print(pandas_df.isnull().values.any())
    df = pandas_df.set_index(index, inplace=False)
    prices = df.loc[:,prices_start:prices_end]
    mtl_ret = prices.pct_change().resample(resample_freq).agg(lambda x: (1+x).prod()-1)
    past_11 = (mtl_ret+1).rolling(lookback).apply(np.prod)-1
    return prices, mtl_ret, past_11


In [ ]:
dfff = dff.set_index("Date", inplace = False)
x = dfff.resample("M")
type(x)

In [ ]:
#test out the data_cleaner function

prices, mtl_ret, past_11 = data_cleaner(dff, lookback = 3, resample_freq= 'BM')
print(f'\nprices:\n {prices}, \nmonthly_return:\n {mtl_ret}, \npast_11:\n {past_11}')

In [ ]:
past_11.head(5)

In [ ]:
def momentum(formation, past_11, num_l_s=5):
    end_measurement = formation - MonthEnd(1)
    ret_12 = past_11.loc[end_measurement]
    ret_12 = ret_12.reset_index()
    ret_12.rename(columns={ ret_12.columns[1]: "returns" }, inplace = True)
    ret_12 = ret_12.sort_values(["returns"], ascending=False)
    
    middle = 11-2*num_l_s
    l_s = []
    for i in range(num_l_s):
        l_s.append(2)
    for i in range(middle):
        l_s.append(1)
    for i in range(num_l_s):
        l_s.append(0)
    ret_12["bucket"] = l_s # Make arbitrary number of buckets
    
    winners = ret_12[ret_12.bucket == 2]
    losers = ret_12[ret_12.bucket == 0]
    winnerret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(winners["index"])]
    loserret = mtl_ret.loc[formation + MonthEnd(1), mtl_ret.columns.isin(losers["index"])]
    Momentumprofit = winnerret.mean() - loserret.mean()
    return Momentumprofit







In [ ]:
formation = dt.datetime(1999, 12, 31, 0, 0)
momentum(formation, past_11) # has to be identical with [35]

In [ ]:
def benchmarker(pandas_df, profits_list):
    dff = pandas_df.set_index("Date", inplace=False)
    
    benchmark = dff[["MSCI_EU"]]
    benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
    
    bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
    comparison = pd.DataFrame(bm_mtl_ret)
    comparison["mom_ret"] = profits_list
    comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
    comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
    comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
    print(comparison)
    
    fig, ax = plt.subplots()
    ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
    ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
    ax.legend(loc="best")
    ax.set_ylabel('Monthly Returns')
    ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")


In [ ]:
# Creating loop for every month  starting at formation date -> creates df with all these dates
for i in range(273): # Eig wären es 273 Monate zwischen 31.12.1999 und 31.12.2022 -> stimmt irgendwie ned ganz aber lauft jetzt halt so bis endi august
    print(formation + MonthEnd(i))

#wären es nicht 276 Monate? has mit emene Zeitspannen-rechner usgrechnet --> gahd ets bis endi november 2022
for i in range(276):
    print(formation + MonthEnd(i))


In [ ]:
# Creating loop for storing the profits and the realized dates of the momentum strategy
profits = []
dates = []

'''
for i in range(273):
    profits.append(momentum(formation + MonthEnd(i)))
    dates.append(formation + MonthEnd(i))
'''

#version 2.0 (da gahds nume bis 274 --> gid en error wenn en monet meh nimmsch)
for i in range(274):
    profits.append(momentum(formation + MonthEnd(i), past_11))
    dates.append(formation + MonthEnd(i))


In [ ]:
benchmarker(df, profits)

In [ ]:
mom = pd.DataFrame({"Dates": dates,"Profits": profits})
mom

# Benchmarking

In [ ]:
benchmark = df[["MSCI_EU"]]
benchmark = benchmark.loc["1999-12-31 00:00:00":"2022-09-30 00:00:00"]
benchmark

In [ ]:
bm_mtl_ret = mtl_ret = benchmark.pct_change().resample("M").agg(lambda x: (1+x).prod()-1) # pct_change creates ordinary returns, resample Monthly and aggregating with the (1+x) -1 formula to get monthly ordinary returns
bm_mtl_ret


In [ ]:
#compare momentum result vs benchmark
comparison = pd.DataFrame(bm_mtl_ret)
comparison["mom_ret"] = profits
comparison.rename(columns={ comparison.columns[0]: "bench_ret" }, inplace = True)
comparison["bench_indexed"] = comparison["bench_ret"].add(1).cumprod()
comparison["mom_indexed"] = comparison["mom_ret"].add(1).cumprod()
comparison


In [ ]:
comparison

# Comparison Plot

In [ ]:
# Plotting
fig, ax = plt.subplots()
ax.plot(comparison.index,comparison.mom_indexed, label="Momentum", color="red")
ax.plot(comparison.index,comparison.bench_indexed, label="MSCI EU", color="blue")
ax.legend(loc="best")
ax.set_ylabel('Monthly Returns')
ax.set_title("Momentum long/short 5 sectors monthly rebalancing vs MSCI EU, indexed 31.12.1999")